In [71]:
import pandas as pd

df = pd.read_csv('recipes_data.csv')
df.columns

Index(['title', 'ingredients', 'directions', 'link', 'source', 'NER', 'site'], dtype='object')

In [72]:
import ast
# pd.set_option('display.max_colwidth', None)
df['NER'] = df['NER'].apply(lambda x: set(ast.literal_eval(x)))
df = df[df['NER'] != set()]
display(df)

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"{bite size shredded rice biscuits, nuts, brown...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"{beef, chicken breasts, sour cream, cream of m...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"{pepper, frozen corn, cream cheese, salt, butt...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"{shredded cheese, chicken, cream of mushroom s...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"{graham cracker crumbs, chocolate chips, powde...",www.cookbooks.com
...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"{hazelnuts, tortillas, marshmallows, chocolate...",www.foodnetwork.com
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"{eggs, choice, relish, salt, paprika, miracle ...",cookpad.com
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"{sesame oil, soy sauce, salt, radish, white se...",cookpad.com
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"{coriander seeds, thyme, apples, apple cider, ...",cooking.nytimes.com


In [73]:
all_ingredients = set.union(*df['NER'].tolist())
ingredients_df = pd.DataFrame(list(all_ingredients), columns=['name'])

In [74]:
recipes_df = df.drop(columns=['ingredients', 'directions', 'source', 'NER']).rename(columns={'title': 'name'})
recipes_df['id'] = df.index
recipes_df = recipes_df[['id', 'name', 'link', 'site']]
recipes_df

,id,name,link,site
0,0,No-Bake Nut Cookies,www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com
1,1,Jewell Ball'S Chicken,www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com
2,2,Creamy Corn,www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com
3,3,Chicken Funny,www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com
4,4,Reeses Cups(Candy),www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com
...,...,...,...,...
2231137,2231137,Sunny's Fake Crepes,www.foodnetwork.com/recipes/sunny-anderson/sun...,www.foodnetwork.com
2231138,2231138,Devil Eggs,cookpad.com/us/recipes/355411-devil-eggs,cookpad.com
2231139,2231139,Extremely Easy and Quick - Namul Daikon Salad,cookpad.com/us/recipes/153324-extremely-easy-a...,cookpad.com
2231140,2231140,Pan-Roasted Pork Chops With Apple Fritters,cooking.nytimes.com/recipes/1015164,cooking.nytimes.com


In [75]:
recipe_ingredients_df = df.explode('NER')
recipe_ingredients_df['recipe_id'] = recipe_ingredients_df.index
recipe_ingredients_df = recipe_ingredients_df.drop(columns=['title', 'ingredients', 'directions', 'link', 'source', 'site']).rename(columns={'NER': 'ingredient'}).reset_index()
recipe_ingredients_df = recipe_ingredients_df[['recipe_id', 'ingredient']]
recipe_ingredients_df

,recipe_id,ingredient
0,0,bite size shredded rice biscuits
1,0,nuts
2,0,brown sugar
3,0,milk
4,0,vanilla
...,...,...
18420498,2231141,egg whites
18420499,2231141,flat-leaf
18420500,2231141,tomato paste
18420501,2231141,ground veal


In [76]:
# ingredients_to_check = {"pepper", "brown rice"}
# mask = df['NER'].apply(lambda x : ingredients_to_check.issubset(set(x)))
# matching_rows = df[mask]

# res = (matching_rows['NER'])
# display(res)